In [ ]:
# This notebook is about making wordcloud of tweets using python

In [ ]:
# the original analysis has 3000 tweets, but since I've already scrpaed a lot for trial and error, I could only get ~300
# used a lot of links to correct course during this code
# https://medium.com/codex/making-wordcloud-of-tweets-using-python-ca114b7a4ef4 <<< original article
# https://github.com/vijayv500/Machine-Learning/blob/main/Wordcloud_tweets.ipynb <<< original github code 
# << have modified this in places using:
# https://stackoverflow.com/questions/69332653/tweepy-why-did-i-receive-attributeerror-for-search <<< search word modified
# https://stackoverflow.com/questions/7465590/nameerror-name-re-is-not-defined <<< re was not imported
# https://www.geeksforgeeks.org/generate-word-clouds-of-any-shape-in-python/ <<< had to include r to import wordcloud image
# https://i.stack.imgur.com/M2jeo.jpg <<< image used for wordcloud

In [ ]:
!pip install tweepy

In [ ]:
import tweepy
import pandas as pd

In [ ]:
# add your twitter developer codes here
# follow these steps: https://youtu.be/wlnx-7cm4Gg?t=162   <<< starts at 2:42 for how to apply, see for only 2-3 mins
# This is where the codes are generated: https://youtu.be/3353Mgdme4c?t=770  << at 12 mins~
# These are the links to apply, comes in 2-3 days
# https://developer.twitter.com/  get your essential access here
# https://developer.twitter.com/en/portal/products/elevated get your elevated access here

In [ ]:
# Authentication
consumerKey = "your key"
consumerSecret = "your key"
accessToken = "your key"
accessTokenSecret = "your key"

In [ ]:
auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

In [ ]:
def keyword_to_csv(keyword,recent):
    try:
        tweets = tweepy.Cursor(api.search_tweets,q=keyword).items(recent) #creates query method
        tweets_list = [[tweet.text] for tweet in tweets]
#pulls text information from tweets
        df = pd.DataFrame(tweets_list,columns=['Text']) 
#creates a pandas dataframe
        df.to_csv('{}.csv'.format(keyword), sep=',', index = False)
 #creates a csv from data frame
    except BaseException as e:
        print('failed on_status,',str(e))
        time.sleep(3)

In [ ]:
keyword = '#Omicron'+ "-filter:retweets"
recent  = 300
keyword_to_csv(keyword, recent)

In [ ]:
df = pd.read_csv("./#Omicron-filter:retweets.csv")
pd.options.display.max_colwidth = 200
df.head()

In [ ]:
df.shape

In [ ]:
 #loads the row from dataframe
a = df.loc[299].to_string()
print(a)

In [ ]:
import re

In [ ]:
re_list = ['(https?://)?(www\.)?(\w+\.)?(\w+)(\.\w+)(/.+)?', '@[A-Za-z0-9_]+','#']
combined_re = re.compile( '|'.join( re_list) )

In [ ]:
regex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)

In [ ]:
a = df.loc[0].to_string()
print(a)

In [ ]:
pattern = re.compile(r'(https?://)?(www\.)?(\w+\.)?(\w+)(\.\w+)(/.+)?')
match = re.sub(pattern,'',a)
print(match)

In [ ]:
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
token = WordPunctTokenizer()

In [ ]:
def cleaning_tweets(t):
    del_amp = BeautifulSoup(t, 'lxml')
    del_amp_text = del_amp.get_text()
    del_link_mentions = re.sub(combined_re, '', del_amp_text)
    del_emoticons = re.sub(regex_pattern, '', del_link_mentions)
    lower_case = del_emoticons.lower()
    words = token.tokenize(lower_case)
    result_words = [x for x in words if len(x) > 2]
    return (" ".join(result_words)).strip()

In [ ]:
print("Cleaning the tweets...\n")
cleaned_tweets = []
for i in range(0,299):
    if( (i+1)%100 == 0 ):
        print("Tweets {} of {} have been processed".format(i+1,299))
    cleaned_tweets.append(cleaning_tweets((df.Text[i])))

In [ ]:
string = pd.Series(cleaned_tweets).str.cat(sep=' ')

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

stopwords = set(STOPWORDS)
stopwords.update(["Omicron","Omnicron","omicron","variant"])

In [ ]:
wordcloud = WordCloud(width=1600, stopwords=stopwords,height=800,max_font_size=200,max_words=50,collocations=False, background_color='grey').generate(string)
plt.figure(figsize=(40,30))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
import numpy as np
from PIL import Image

In [ ]:
mask = np.array(Image.open(r'C:\tweet.jpg'))

In [ ]:
wordcloud = WordCloud(width=1600, mask = mask,stopwords=stopwords,height=800,max_font_size=200,max_words=50,collocations=False,background_color='white').generate(string)

In [ ]:
f = plt.figure(figsize=(50,50))
f.add_subplot(1,2, 1)
plt.imshow(mask, cmap=plt.cm.gray, interpolation='bilinear')
plt.title('Original Image', size=30)
plt.axis("off")
f.add_subplot(1,2, 2)
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Generated Word Cloud', size=30)
plt.axis("off")
plt.show()